# 1. Apply run length encoding for the following string and compress it.


In [ ]:
s='eeeeeeerrrrrrrrrrrrrrrrttttttttttttttiiiiiiiifffffeft'
from collections import OrderedDict
enc=OrderedDict.fromkeys(s, 0)
for i in s:
  enc[i] += 1
  
encoded = ''
for key,value in enc.items():
  encoded = encoded + key + str(value)

encoded


'e8r16t15i8f6'

## 2.Inverted Index File with Terms, Occurrences and Posting List


In [ ]:
dict={'Samsung':[233,[2, 12, 34544, 34574, 35569]],'Airtel':[12,[12, 17, 25, 148, 156, 159, 172]],'Mercury':[15,[1, 2, 3, 7, 9, 10]],'Venus':[12,[23, 45, 78, 122, 145]],'Fiber':[6,[1, 3, 5, 7, 19, 20]]}
dict

{'Airtel': [12, [12, 17, 25, 148, 156, 159, 172]],
 'Fiber': [6, [1, 3, 5, 7, 19, 20]],
 'Mercury': [15, [1, 2, 3, 7, 9, 10]],
 'Samsung': [233, [2, 12, 34544, 34574, 35569]],
 'Venus': [12, [23, 45, 78, 122, 145]]}

#### Apply Binary coding for term “Mercury” (apply for all doc ids)


In [ ]:
print('Before encoding: Mercury',dict['Mercury'])
for j in range(len(dict['Mercury'][1])):
  b=bin(dict['Mercury'][1][j])
  dict['Mercury'][1][j]=b[2:]
  
print('After encoding: Mercury',dict['Mercury'])

Before encoding: Mercury [15, [1, 2, 3, 7, 9, 10]]
After encoding: Mercury [15, ['1', '10', '11', '111', '1001', '1010']]


#### Apply Unary coding for term “Fiber”


In [ ]:
print('Before encoding: Fiber',dict['Fiber'])
for j in range(len(dict['Fiber'][1])):
  b=dict['Fiber'][1][j]
  c=''
  for i in range(b+1):
    if i == b:
      c+=str(1)
    else:
      c+=str(0)
  dict['Fiber'][1][j]=c
  
print('After encoding: Fiber',dict['Fiber'])

Before encoding: Fiber [6, [1, 3, 5, 7, 19, 20]]
After encoding: Fiber [6, ['01', '0001', '000001', '00000001', '00000000000000000001', '000000000000000000001']]


#### Apply Elias Gamma Encoding for term “Airtel”


In [ ]:
from math import log
log2=lambda x:log(x,2)
def Unary(x):
    return (x-1)*'0'+'1'

def Binary(x, l=1):
    s='{0:0%db}'%l
    return s.format(x)
      
def Elias_Gamma(x):
    if(x==0): 
        return '0'
  
    n=1+int(log2(x))
    b=x-2**(int(log2(x)))
  
    l=int(log2(x))
  
    return Unary(n)+Binary(b,l)

In [ ]:
print('Before encoding: Airtel',dict['Airtel'])
for j in range(len(dict['Airtel'][1])):
  b=Elias_Gamma(dict['Airtel'][1][j])
  dict['Airtel'][1][j]=b
  
print('After encoding: Airtel',dict['Airtel'])

Before encoding: Airtel [12, [12, 17, 25, 148, 156, 159, 172]]
After encoding: Airtel [12, ['0001100', '000010001', '000011001', '000000010010100', '000000010011100', '000000010011111', '000000010101100']]


#### Apply Elias Delta Decoding for “000010000”


In [ ]:
s='000010000'
import math
  
def  del_dec(x):
    x = list(x)
    i=0
    while True:
        if not x[i] == '0':
            break
        i+=1
    x=x[2*i+1:] 
    x.insert(0,'1') 
    x.reverse()
    n=0
    for i in range(len(x)): 
        if x[i]=='1':
            n=n+math.pow(2,i)
    return int(n)
  
del_dec(s)

1

#### Apply Elias Delta Encoding for term “Venus”

In [ ]:
from math import floor
def no_msb(x):
    binary = "{0:b}".format(int(x))
    bin_no_msb = binary[1:]
    return bin_no_msb

def gam_enc(k):
    if (k == 0):
        return '0'
    N = 1 + floor(log(k, 2))
    Unary = (N-1)*'0'+'1'
    return Unary + no_msb(k)

def del_enc(x):
    Gamma = gam_enc(1 + floor(log(x, 2)))
    binary_without_MSB = no_msb(x)
    return Gamma+binary_without_MSB

In [ ]:
print('Before encoding: Venus',dict['Venus'])
for j in range(len(dict['Venus'][1])):
  b=del_enc(dict['Venus'][1][j])
  dict['Venus'][1][j]=b
  
print('After encoding: Venus',dict['Venus'])

Before encoding: Venus [12, [23, 45, 78, 122, 145]]
After encoding: Venus [12, ['001010111', '0011001101', '00111001110', '00111111010', '00010000010001']]


#### Apply Elias Delta Decoding for “00101001”


In [ ]:
del_dec('00101001')

9

#### Apply Variable Byte Encoding for “Samsung”. (Use doc ids gap)

In [ ]:
from __future__ import division 
from struct import pack, unpack
def encode_number(number):
    bytes_list = []
    while True:
        bytes_list.insert(0, number % 128)
        if number < 128:
            break
        number = number // 128
    bytes_list[-1] += 128
    return pack('%dB' % len(bytes_list), *bytes_list)

def encode(numbers):
    bytes_list = []
    for number in numbers:
        bytes_list.append(encode_number(number))
    return b"".join(bytes_list)

In [ ]:
dict={'Samsung':[233,[2, 12, 34544, 34574, 35569]],'Airtel':[12,[12, 17, 25, 148, 156, 159, 172]],'Mercury':[15,[1, 2, 3, 7, 9, 10]],'Venus':[12,[23, 45, 78, 122, 145]],'Fiber':[6,[1, 3, 5, 7, 19, 20]]}

In [ ]:
print("Variable byte encoding for 'Samsung' is:",encode(dict['Samsung'][1]))

Variable byte encoding for 'Samsung' is: b'\x82\x8c\x02\r\xf0\x02\x0e\x8e\x02\x15\xf1'
